In [22]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost
from xgboost import XGBClassifier
import os
import joblib

In [23]:
folder_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\html.2023.final.data\release"
rain_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案\extend_X_feature\weather\If_rain.xlsx"
holiday_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案\extend_X_feature\holiday\If_holiday.xlsx"
all_the_stops_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\html.2023.final.data\demographic.json"

In [24]:
with open(all_the_stops_path, 'r', encoding='utf-8') as json_file:
    parsed_data = json.load(json_file)
    all_the_stops = list(parsed_data.keys())

In [25]:
def read_feature(feature_path, feature_name):#讀取feature資料，資料我自己編製的excel檔案
    df = pd.read_excel(feature_path)
    df_dict = {}
    for index, row in df.iterrows():
        key = int(row['date'])
        value = row[feature_name]
        df_dict[str(key)] = value
    return df_dict

In [26]:
def read_json_files(folder_path, specific_bike_stop, rain_data, holiday_data):#讀老師給的檔案，怎麼讀的不太重要，因為要看資料的邏輯，所以知道是讀資料就好
    Date = []#注意參數stop_point我指的是讀幾個車站的意思，你要輸出五個車站就打五
    Data = []#rain_data跟holiday_data都是我目前先用很硬幹的方法輸入這個function，你可以參考我的feature然後做一個feature的excel檔案給我，我明天讀

    
    for root, dirs, files in os.walk(folder_path):
        
        if str(root)[-8:][0] != '2':
            continue
        
        for file in files:
            #print(str(file))
            if file.endswith('.json') and str(file)[:9] == specific_bike_stop:
                #print(f'{str(root)[-8:]} :yes')
                file_path = os.path.join(root, file)
  
                with open(file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)
                    
                    for time, value in data.items():
                        if value:
                            entry = {
                                'date': str(root)[-8:],
                                'bike_stop': str(file)[:9],
                                'time': time,
                                'total': value["tot"],
                                'current': value["sbi"],
                                'can_park': value["bemp"],
                                'open': value["act"],
                                'rain_hour': rain_data[f'{str(root)[-8:]}'],
                                'holiday': holiday_data[f'{str(root)[-8:]}']
                            }
                            Data.append(entry)
        Date.append({str(root)[-8:]})
    
    return Data, Date, 

In [27]:
def time_to_minutes(time_str):#時間是str不能丟train，我改成用總分鐘數，他就會遞增然後是int
    hour, minute = map(int, time_str.split(':'))
    return hour * 60 + minute

In [28]:
def err(y_test, y_pred, total_stop):#老師講義的error function，照打而已
    sum = 0
    for i in range(y_test.size):
        yt = y_test[i]
        yp = y_pred[i]
        sum += 3*(abs(yt-yp)/total_stop)*(abs(yt/total_stop-1/3)+abs(yt/total_stop-2/3))
    sum /= y_test.size
    return sum

In [29]:
best_models = {}

In [30]:
def find_missing(y_train):
    num_elem = len(set(y_train))
    expected_set = set(range(num_elem))
    #print(expected_set)
    #print(set(y_train))
    missing_num = expected_set - set(y_train)
    wrong_num = set(y_train) - expected_set
    if missing_num and wrong_num:
        missing_num_int = next(iter(missing_num))
        wrong_num_int = next(iter(wrong_num))
        return missing_num_int, wrong_num_int
    else:
        return 0, 0

In [31]:
def trainModels(new_data, best_models):
    for stop in new_data:#這裡stop指的是車站名稱
        print("Bike stop ID: ", stop)
        X = []
        y = []
        total_stop = 0
        for elem in new_data[stop]:#這裡開始把資料轉為Xy形式，簡單來說，讓機器看得懂，理論上一個車站X.shape -> (very big, 5), 前面這個5是feature的數量， y.shape -> (very big), 這very big值相同 主要跟你讀進幾個車站的資料成正比
            total_stop = elem.get('total')
            if elem.get('open') == 0:
                continue
            X.append([time_to_minutes(elem.get('time')), elem.get('total'), elem.get('open'), elem.get('rain_hour'), elem.get('holiday')])
            y.append(elem.get('current'))
        X = np.array(X)
        #print(X.shape)
        y = np.array(y)
        X_numeric = X.astype(float)
        smallest_error = float('inf')
        best_model = XGBClassifier(n_estimators=100, max_depth = 10, random_state=42) #initialize 隨機森林的model
        for i in range(1):#我做十次，用不同的資料切割跑，使用最小的error的那個
            X_train, X_val, y_train, y_val = train_test_split(X_numeric, y, test_size=0.1, random_state=42)#資料的切割，這公式很好用 0.1是指切多少當validation，每跑一次用跟前一次不同的亂數種，注意亂數種不用每次compile都不同，因為想比較演算法能不能在同資料下變更好
            
            model = XGBClassifier(n_estimators= 200, learning_rate= 0.1)
            
            #500108040 發生奇怪問題的修正
            missing_num, wrong_num = find_missing(y_train)
            y_train_modified = [missing_num if label == wrong_num else label for label in y_train]
            model.fit(X_train, y_train_modified)
            
            y_pred = model.predict(X_val)
            #print(y_test)
            #print(y_pred)  
            #for true_value, predicted_value in zip(y_test, y_pred):
                #print(f"True: {true_value}, Predicted: {predicted_value}")
            #print("正確率", model.score(X_val,y_val))
            error = err(y_val, y_pred, total_stop)
            if error < smallest_error:
                smallest_error = error
                best_model = model
                
        best_models[str(stop)] = best_model
        print(smallest_error)



In [32]:
def transformData(data, specific_bike_stop):
    new_data = {}
    new_data[specific_bike_stop] = []

    for elem in data:#原先資料讀進來是某日->車站->data, 我改成車站->某日->data。簡單來說，是要讓每個車站都有一個model，資料要長這樣比較好讀
        bike_stop = elem.get('bike_stop')
        if bike_stop == specific_bike_stop:
            new_data[bike_stop].append(elem)
    return new_data

In [33]:
rain_data = read_feature(rain_path, 'rain_hour')
holiday_data = read_feature(holiday_path, "holiday")

In [34]:
import os
import joblib
def save_model(key, model):
    # Create a folder to store the models if it doesn't exist
    custom_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案\models"
    os.makedirs(custom_path, exist_ok=True)
    # Iterate through the models and store them in the folder
    for key, model in best_models.items():
        model_filename = os.path.join(custom_path, f'model_{str(key)}.pkl')
        joblib.dump(model, model_filename)

In [35]:
all_the_stops = all_the_stops[950:]
print(len(all_the_stops))
for specific_bike_stop in all_the_stops:
    data, date = read_json_files(folder_path, specific_bike_stop, rain_data, holiday_data)
    new_data = transformData(data, specific_bike_stop)
    trainModels(new_data, best_models)
    save_model(specific_bike_stop, best_models[specific_bike_stop])

374
Bike stop ID:  500110023
0.15348935494566696
Bike stop ID:  500110024
0.16518227286099912
Bike stop ID:  500110025
0.17763228507030293
Bike stop ID:  500110026
0.2210720187336318
Bike stop ID:  500110027
0.26664144916572025
Bike stop ID:  500110028
0.14127508929204052
Bike stop ID:  500110029
0.19097528684907325
Bike stop ID:  500110030
0.14369831064044272
Bike stop ID:  500110031
0.10583054925197544
Bike stop ID:  500110032
0.20609759172865638
Bike stop ID:  500110033
0.21497817656981189
Bike stop ID:  500110034
0.20758731559702315
Bike stop ID:  500110035
0.2196006885921682
Bike stop ID:  500110036
0.18583596189308896
Bike stop ID:  500110037
0.17333122006651705
Bike stop ID:  500110038
0.18380406001764937
Bike stop ID:  500110039
0.12780394164979283
Bike stop ID:  500110040
0.15603328710124675
Bike stop ID:  500110041
0.20791009960912527
Bike stop ID:  500110042
0.20157105427349978
Bike stop ID:  500110043
0.17574244525700844
Bike stop ID:  500110044
0.14742790816353055
Bike sto

KeyboardInterrupt: 

In [ ]:
custom_path = r"C:\Users\Tim Chen\OneDrive\桌面\台大\台大機器學習\機器學習專案\models"

# Get a list of all files in the folder
model_files = [file for file in os.listdir(custom_path) if file.endswith('.pkl')]

# Load each model from the files
best_models_models = {}
for model_file in model_files:
    # Extract the key from the file name (assuming the file name is like 'model_key.pkl')
    key = str(model_file[6:15])
    # Load the model
    model_path = os.path.join(custom_path, model_file)
    loaded_model = joblib.load(model_path)
    
    # Store the loaded model in the dictionary
    best_models[key] = loaded_model

# Now 'loaded_models' contains all your models
print(best_models)

{'500101001': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...), '500101002': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
       

In [ ]:
def find_total_stops(specific_bike_stop):
    for root, dirs, files in os.walk(folder_path):
        
        if str(root)[-8:][0] != '2':
            continue
        
        for file in files:
            #print(str(file))
            if file.endswith('.json') and str(file)[:9] == specific_bike_stop:
                #print(f'{str(root)[-8:]} :yes')
                file_path = os.path.join(root, file)
  
                with open(file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)
                    
                    for time, value in data.items():
                        if value:
                            return value["tot"]

In [ ]:
import csv
predict_path = r'C:\Users\ET\Desktop\機器學習專案\html.2023.final.data\sample_submission_stage2.csv'
df = pd.read_csv(predict_path)
predict_time = df['id'].tolist()

weather_forecast_path = r'C:\Users\ET\Desktop\機器學習專案\ML\extend_X_feature\predict_date_rain_hour\predict_rain_hour.xlsx'
pred_rain = read_feature(weather_forecast_path, "rain_hour")
the_holiday = read_feature(weather_forecast_path, "holiday")

my_prediction = []
timer = 0

for elem in predict_time:
    timer += 1
    x_to_pred = []
    future_date = elem[:8]
    stop_to_predict = elem[9:18]
    time_to_predict = time_to_minutes(elem[19:])
        
    rain_hour = pred_rain[future_date]
    holiday = the_holiday[future_date]
    
    x_to_pred.append([time_to_predict, find_total_stops(stop_to_predict), 1, rain_hour, holiday])
    y_pred = best_models[stop_to_predict].predict(x_to_pred)
    my_prediction.append(y_pred)
    
    if timer == 72:
        timer = 0
        print(f'{stop_to_predict}, {future_date}')

500101001, 20231021
500101002, 20231021
500101003, 20231021
500101004, 20231021
500101005, 20231021
500101006, 20231021
500101007, 20231021
500101008, 20231021
500101009, 20231021
500101010, 20231021
500101013, 20231021
500101014, 20231021
500101015, 20231021
500101018, 20231021
500101019, 20231021
500101020, 20231021
500101021, 20231021
500101022, 20231021
500101023, 20231021
500101024, 20231021
500101025, 20231021
500101026, 20231021
500101027, 20231021
500101028, 20231021
500101029, 20231021
500101030, 20231021
500101031, 20231021
500101032, 20231021
500101033, 20231021
500101034, 20231021
500101035, 20231021
500101036, 20231021
500101037, 20231021
500101038, 20231021
500101039, 20231021
500101040, 20231021
500101041, 20231021
500101042, 20231021
500101091, 20231021
500101092, 20231021
500101093, 20231021
500101094, 20231021
500101114, 20231021
500101115, 20231021
500101123, 20231021
500101166, 20231021
500101175, 20231021
500101176, 20231021
500101181, 20231021
500101184, 20231021


In [ ]:
import joblib
def find_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

In [ ]:
print(type(my_prediction[1320]))

<class 'numpy.ndarray'>


In [ ]:
import openpyxl
# Specify the Excel file name to save
excel_filename = "my_prediction.xlsx"

# Create a new Excel workbook
wb = openpyxl.Workbook()

# Select the active sheet
ws = wb.active

# Write the 1D array data to a single column in the Excel sheet
for item in my_prediction:
    ws.append(list(item))

# Save the Excel workbook to a file
directory_path = r"C:\Users\ET\Desktop\機器學習專案"

excel_filepath = os.path.join(directory_path, excel_filename)
wb.save(excel_filepath)